In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from nilearn import image
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix, plot_stat_map
from nilearn.interfaces.fmriprep import load_confounds
sys.path.append('..')
from utils.data import Subject, load_participant_list
from utils.analysis import compute_parametric_modulator

In [3]:
base_dir = '/home/ubuntu/data/learning-habits'
bids_dir = '/home/ubuntu/data/learning-habits/bids_dataset/derivatives/fmriprep-23.2.1/'

sub_ids = load_participant_list(base_dir)

In [4]:
#subjects = [Subject(base_dir, sub_id, include_modeling=True, include_imaging=True, bids_dir=bids_dir) for sub_id in sub_ids]
subject = Subject(base_dir, '01', include_modeling=True, include_imaging=True, bids_dir=bids_dir)

In [5]:
run='learning2'
block = getattr(subject, run)
events = block.extend_events_df()

In [6]:
block.extended_trials

,left_stim,right_stim,left_value,right_value,shift,action,rt,chosen_stim,reward,correct,...,stim6_value_ck,stim7_value_ck,stim8_value_ck,value_diff,first_stim,second_stim,first_stim_value_rl,second_stim_value_rl,first_stim_value_ck,second_stim_value_ck
trial,,,,,,,,,,,,,,,,,,,,,
1,8,7,5,4,0,NaN,NaN,NaN,NaN,NaN,...,0.049835,0.101242,0.132941,NaN,7,8,3.988659,4.982170,0.101242,0.132941
2,3,1,2,1,0,1.0,0.409929,3.0,2.0,1.0,...,0.049835,0.101242,0.132941,1.020078,1,3,1.003978,1.996785,0.043020,0.070290
3,5,7,3,4,1,2.0,0.597076,7.0,4.0,1.0,...,0.049835,0.101242,0.132941,-1.055042,5,7,2.991069,3.988659,0.043790,0.101242
4,8,6,5,4,0,1.0,0.564293,8.0,5.0,1.0,...,0.049835,0.107348,0.132941,1.072800,6,8,3.992477,4.982170,0.049835,0.132941
5,3,5,2,3,0,2.0,0.599684,5.0,3.0,1.0,...,0.049496,0.107348,0.138832,-0.962337,5,3,2.992893,1.997442,0.043493,0.076607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,3,1,2,1,1,1.0,0.571575,3.0,2.0,1.0,...,0.074222,0.201062,0.248599,1.118596,3,1,1.999983,1.000021,0.161379,0.042745
93,2,4,2,3,1,2.0,0.748851,4.0,3.0,1.0,...,0.074222,0.201062,0.248599,-1.101046,2,4,1.999951,2.999915,0.082045,0.183127
94,8,6,5,4,0,1.0,0.724958,8.0,5.0,1.0,...,0.074222,0.201062,0.248599,1.174279,6,8,3.999951,4.999853,0.074222,0.248599


In [7]:
events_df = block.extend_events_df(columns_event={'first_stim': 'first_stim_presentation'})
events_df

,onset,duration,trial_type,trial,first_stim
0,0.001868,0.820289,first_stim_presentation,1,7.0
1,0.822156,1.000000,second_stim_presentation,1,0.0
2,1.822156,1.504399,non_response_feedback,1,0.0
3,3.326556,6.266152,iti,1,0.0
4,9.592707,0.848439,first_stim_presentation,2,1.0
...,...,...,...,...,...
567,959.462614,0.623545,second_stim_presentation,96,0.0
568,960.086159,0.000000,response,96,0.0
569,960.087956,1.879011,purple_frame,96,0.0
570,960.589006,1.377960,points_feedback,96,0.0


In [8]:
# let's create a function that transforms the events df by splitting the trial_type first_stim_presentation
def transform_events_df(events_df):
    events_df['trial_type'] = events_df.apply(
        lambda row: f"{row['trial_type']}_{int(row['first_stim'])}" if row['trial_type'] == 'first_stim_presentation' else row['trial_type'],
        axis=1
    )
    return events_df

events_df = transform_events_df(events_df)
events_df

,onset,duration,trial_type,trial,first_stim
0,0.001868,0.820289,first_stim_presentation_7,1,7.0
1,0.822156,1.000000,second_stim_presentation,1,0.0
2,1.822156,1.504399,non_response_feedback,1,0.0
3,3.326556,6.266152,iti,1,0.0
4,9.592707,0.848439,first_stim_presentation_1,2,1.0
...,...,...,...,...,...
567,959.462614,0.623545,second_stim_presentation,96,0.0
568,960.086159,0.000000,response,96,0.0
569,960.087956,1.879011,purple_frame,96,0.0
570,960.589006,1.377960,points_feedback,96,0.0
